In [1]:
import torch.nn as nn
import torch
from torch.autograd import Variable
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
import torch.nn.functional as F
from data_preparator import *
from torch.utils.data import Dataset, DataLoader

class DAC(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_size):
        super(DAC, self).__init__()
        self.hidden_size = hidden_size
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        self.embedding = nn.Embedding(self.vocab_size+1, self.embedding_dim)
        self.gru = nn.GRU(embedding_dim, hidden_size, bidirectional=True)
        self.fc = nn.Linear(hidden_size, 7)

    def forward(self, seq, lengths, gpu = True):
        print("Sequence shape: ", seq.shape)
        bs = seq.size(1)
        print("Batch size: ", bs)
        self.hidden = self._init_hidden(bs, gpu)
        
        embeds = self.embedding(seq)
        embeds = pack_padded_sequence(embeds, lengths) #inpad
        
        gru_out, self.hidden = self.gru(embeds, self.hidden) #retorna o hidden_state de todos os timesteps
        
        gru_out, lenghts = pad_packed_sequence(gru_out) # faz o pad da sequencia para o tamanho maximo do batch
        
        print('GRU output(all timesteps)', gru_out.shape)
        print(gru_out)
        
        output = self.fc(self.hidden[-1])
        
        return F.log_softmax(output, dim=-1)
    
    def init_hidden(self, batch_size, gpu):
        if gpu: return Variable(torch.zeros((1,batch_size,self.hidden_size)).cuda())
        else: return Variable(torch.zeros((1,batch_size,self.hidden_size)))
        return self.create_variable(hidden)

    def create_variable(self, tensor):
        # Do cuda() before wrapping with variable
        if torch.cuda.is_available():
            return Variable(tensor.cuda())
        else:
            return Variable(tensor)

/home/joao/virtualenvs/esquizo/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/joao/virtualenvs/esquizo/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [ ]:
padded, labels, vocab_size = prepare_dataset('./conversas_mexidas.csv')

In [ ]:
hidden_size = 30

In [32]:
m = DAC(vocab_size, 20, hidden_size)

print(m)

[autoreload of prompt_toolkit failed: Traceback (most recent call last):
  File "/home/joao/virtualenvs/esquizo/lib/python3.6/site-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/home/joao/virtualenvs/esquizo/lib/python3.6/site-packages/IPython/extensions/autoreload.py", line 368, in superreload
    module = reload(module)
  File "/home/joao/virtualenvs/esquizo/lib/python3.6/imp.py", line 315, in reload
    return importlib.reload(module)
  File "/home/joao/virtualenvs/esquizo/lib/python3.6/importlib/__init__.py", line 166, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 618, in _exec
  File "<frozen importlib._bootstrap_external>", line 678, in exec_module
  File "<frozen importlib._bootstrap>", line 219, in _call_with_frames_removed
  File "/home/joao/virtualenvs/esquizo/lib/python3.6/site-packages/prompt_toolkit/__init__.py", line 18, in <module>
    from .shortcuts import 

DAC(
  (embedding): Embedding(2492, 20)
  (gru): GRU(20, 30, bidirectional=True)
  (fc): Linear(in_features=30, out_features=7, bias=True)
)
